## Guitar amplifier emulation research

In [ ]:
import numpy as np
import seaborn as sns
import keras
import keras.backend as K
import matplotlib.pyplot as plt
import librosa
import tensorflow as tf
import IPython

Verify GPU support

In [ ]:
tf.config.list_physical_devices()

Defining required constants

In [ ]:
INPUT_PATH = "data/in.wav"
OUTPUT_PATH = "data/out.wav"

TEST_INPUT_PATH = "data/in.wav"
TEST_OUTPUT_PATH = "data/out.wav"

TRAIN_PER_EPOCH = 15 # seconds
VAL_DURATION = 10 # seconds

BATCH_SIZE = 1
INFERENCE_BATCH_SIZE = 256

INPUT_SIZE = 441
MODEL_NAME = input()

Creating dataset from audio

In [ ]:
from dataset import RawWaveformLoader

train, valid, _ = RawWaveformLoader.DataSplitter(input_path=INPUT_PATH, 
                                                  output_path=OUTPUT_PATH, 
                                                  train_per_epoch=TRAIN_PER_EPOCH, 
                                                  valid_time=VAL_DURATION, 
                                                  batch_size=BATCH_SIZE,
                                                  input_size=INPUT_SIZE,
                                                  dynamic_train=False).split()

demo = RawWaveformLoader(input_path=TEST_INPUT_PATH, 
                         output_path=TEST_OUTPUT_PATH, 
                         batch_size=INFERENCE_BATCH_SIZE,
                         input_size=INPUT_SIZE, 
                         interval_duration=10, 
                         start_offset=86,
                         end_offset=96)

In [ ]:
import keras.backend as K
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Add, BatchNormalization, TimeDistributed

from metrics import error_to_signal

LEARNING_RATE = 0.001
HIDDEN_UNITS = 92

K.clear_session()
input_layer = Input(batch_shape=(BATCH_SIZE, INPUT_SIZE, 1))
rnn = LSTM(HIDDEN_UNITS, return_sequences=True)(input_layer)
rnn = BatchNormalization()(rnn)
rnn = Dense(1)(rnn)
added = Add()([input_layer, rnn])

model = Model(inputs=[input_layer], outputs=added)

model.summary()

model.compile(optimizer=Adam(LEARNING_RATE), loss='mse', metrics=['mse', error_to_signal, 'mae'])

Let's define callbacks

In [ ]:
from keras.callbacks import BackupAndRestore, CSVLogger, TensorBoard, ReduceLROnPlateau

from callbacks import CheckpointAndSave
    
callbacks = [
  BackupAndRestore(f'{MODEL_NAME}/backup'),
  # CheckpointAndSave(f'{MODEL_NAME}/mse_checkpoint', demo, model, BATCH_SIZE, save_best_only=True, save_weights_only=True, monitor='val_mse', verbose=1),
  # CheckpointAndSave(f'{MODEL_NAME}/ets_checkpoint', demo, model, BATCH_SIZE, save_best_only=True, save_weights_only=True, monitor='val_error_to_signal', verbose=1),
  # CheckpointAndSave(f'{MODEL_NAME}/mae_checkpoint', demo, model, BATCH_SIZE, save_best_only=True, save_weights_only=True, monitor='val_mae', verbose=1),
  # ReduceLROnPlateau(patience=10, factor=0.5, cooldown=10, monitor='loss', min_lr=1e-5),
  TensorBoard(log_dir=f'{MODEL_NAME}/logs'),
  CSVLogger(f'{MODEL_NAME}/logs.csv', append=True),
]

In [ ]:
model.fit(train[0][0], train[0][1], batch_size=BATCH_SIZE, shuffle=True, epochs=10000, callbacks=callbacks)

In [ ]:
pred = model.predict(train, batch_size=BATCH_SIZE)

In [ ]:
from scipy.io import wavfile
wavfile.write('out.wav', 44100, pred.reshape(-1, 1))

In [ ]:
import math
import numpy

norm = math.sqrt(sum(numpy.sum(K.get_value(w)) for w in model.optimizer.weights))
norm